# ECG Data Processing – Merging of Individual Subject Data

This notebook merges individual data from all subjects into pandas dataframes.

In [ ]:
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm

import biopsykit as bp

from cft_analysis.io import load_subject_data_dicts

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = bp.colors.fau_palette
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

palette

## Data Import

In [ ]:
data_path = Path("../../../../Data")
ecg_path = data_path.joinpath("ecg")

data_path = Path("../../../data")
data_path_export = data_path.joinpath("processed")
ecg_path_export = data_path_export.joinpath("ecg")

bp.utils.file_handling.mkdirs([data_path_export, ecg_path_export])

### Load Condition List

In [ ]:
condition_list = bp.io.load_subject_condition_list(data_path.joinpath("condition_list.csv"))

### Load Individual ECG Result Files

* Heart Rate Data
* R Peak Data

In [ ]:
subject_dirs = bp.utils.file_handling.get_subject_dirs(ecg_path, pattern="Vp\w+")

In [ ]:
subject_data_dict_hr, subject_data_dict_rpeaks = load_subject_data_dicts(subject_dirs)

## Data Merging

### Create `MIST` Object

In [ ]:
subphases = {
    "BL": 60,
    "RP_CFI": 120,
    "AT": 240,
    "FB": 0
}

structure = {
    "Pre": None,
    "MIST": {
        "MIST1": subphases,
        "MIST2": subphases,
        "MIST3": subphases,
    },
    "Post": None
}

mist = bp.protocols.MIST(name="CFT", structure=structure)
mist

#### Add Heart Rate Data

In [ ]:
mist.add_hr_data(subject_data_dict_hr, subject_data_dict_rpeaks)

#### Compute *Ensemble Time-Series*

In [ ]:
mist.compute_hr_ensemble(
    "hr_ensemble",
    select_phases=True,
    params={
        "normalize_to": "Pre",
        "select_phases": ["MIST1", "MIST2", "MIST3"]
    }
)

#### Compute *Aggregated Results*

In [ ]:
mist.compute_hr_results(
    "hr_mean",
    resample_sec=False,
    normalize_to=False,
    select_phases=True,
    split_into_subphases=True,
    add_conditions=True,
    params={
        "select_phases": ["MIST1", "MIST2", "MIST3"],
        "split_into_subphases": subphases,
        "add_conditions": condition_list
    }
)

mist.compute_hr_results(
    "hr_mean_normalized",
    resample_sec=False,
    normalize_to=True,
    select_phases=True,
    split_into_subphases=True,
    add_conditions=True,
    params={
        "normalize_to": "Pre",
        "select_phases": ["MIST1", "MIST2", "MIST3"],
        "split_into_subphases": subphases,
        "add_conditions": condition_list
    }
)

#### Compute *HRV*

In [ ]:
from neurokit2 import NeuroKitWarning
import warnings
warnings.filterwarnings("ignore", category=NeuroKitWarning)

mist.compute_hrv_results(
    "hrv_phases",
    add_conditions=True,
    params={
        "add_conditions": condition_list
    }
)

mist.compute_hrv_results(
    "hrv_subphases",
    split_into_subphases=True,
    select_phases=True,
    add_conditions=True,
    params={
        "split_into_subphases": subphases,
        "select_phases": ["MIST1", "MIST2", "MIST3"],
        "add_conditions": condition_list
    }
)

### Export Data

In [ ]:
mist.to_file(data_path.joinpath("mist_cft.json"))

In [ ]:
mist.export_hr_results(ecg_path_export)
mist.export_hr_ensemble(ecg_path_export)
mist.export_hrv_results(ecg_path_export)